In [3]:
import os
import pandas as pd
import numpy as np

#-------------------Inputs-------------------------------------------------------------
paddler_mass = 81.64 # Max paddler mass [kg]
g = 9.81 # Gravity [m/s^2]
paddler_weight = paddler_mass*g # Max paddler weight [N]
safety_factor = 1.2
fac_weight = safety_factor*paddler_weight # Factored Weight [N]
A = 50 # Punching area length [mm]
B = 50 # Punching area width [mm]
layer1 = 1 # if layer exists, it equals to 1. Otherwise, zero.
layer2 = 0 # if layer exists, it equals to 1. Otherwise, zero.
d_mesh_1 = 2 # distance b/w the first layer of reinforcement to the surface [mm]
d_mesh_2 = 5 # distance b/w the second layer of reinforcement to the surface [mm]
f_c = 4.9 # Compressive strength of concrete [MPa] (TODO: try tensile)
lamda = 0.75 # Factor to account for lower tensile strength of low-density concrete
fi_c = 0.65 # Safety factor for concrete
alpha_s = 4 # As per CSA A23.3-14 CL 13.3.4.1 (13-7)
#--------------------------------------------------------------------------------------

# Set d to the midpoint between the two layers of reinforcement
# If there is no second layer, d = location of the first layer
if layer2 == 0:
    d = d_mesh_1
else:
    d = (d_mesh_1+d_mesh_2)/2

# Calculate the area
# NOTE: This calculation feels wrong (redundant multip and div), double check <?>
b0 = 2*(A+B)+8*d/2 # [mm]
area = d*b0 # [mm^2]

# Ratio of long side (A) to short side (B) of a column section
beta_c = A/B
if beta_c<1:
    print('Warning: Beta_c is smaller than one.')

# Shear stresses resisted
Vc_1 = (1+2/beta_c)*0.2*lamda*fi_c*np.sqrt(f_c)    
Vc_2 = ((alpha_s*d/b0)+0.19)*lamda*fi_c*np.sqrt(f_c)
Vc_3 = 0.38*lamda*fi_c*np.sqrt(f_c)
Vc = min(Vc_1, Vc_2, Vc_3)

# Shear stresses applied
Vf = fac_weight/area

if Vc>=Vf:
    print('The design is good for punching shear.')
else:
    print('The design needs to be re-done for punching shear.')


ModuleNotFoundError: No module named 'pandas'